In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
def normalized_historical_returns(input_df,data_frequency="Day",return_intervals=[1,4]):
    #input_df: Dataframe with daily/weekly/monthly stock price data
    #return_invervals should be in specified in the increment the input_df is
    #returns are normalized to the data_frequence (i.e. daily data is normalized to daily returns, monthly to per month, etc)

    df = input_df.copy()
    columns = df.columns
    returns = pd.DataFrame()

    for i in return_intervals:
        for col in columns:
            returns[f'{col}_{i}{data_frequency}'] = df[col].pct_change(i).add(1).pow(1/i).sub(1)

    return returns